# **PRUEBA TÉCNICA CARREFIVE**

La conocida cadena de supermercados francesa Carrefive decidió hace unos años establecer un sistema de puntos que se obtienen al comprar sus productos, y sirven para abaratar futuras compras. Al comenzar este 2023, habilitaron la opción de comprar estos puntos directamente con dinero en efectivo. 

De todas estas transacciones, cuentan con muchos datos, pero no saben muy bien qué hacer con ellos. ¿Les ayudamos?

### **TABLAS INCLUIDAS**

1. **Cliente**: con información general sobre cada uno - ID, país de procedencia, fecha de nacimiento, fecha de adhesión al programa de puntos, nivel de cliente y si habla español o no.
2. **Marketing**: relaciona cada cliente con su interés o no en recibir información sobre ofertas al correo electrónico.
3. **Obtención de puntos**: especifica cuántos puntos ha obtenido el cliente en una compra, cuándo y con qué producto.
4. **Uso de puntos**: especifica en qué productos ha utilizado el cliente los puntos, cuántos y cuándo efectuó la compra.
5. **Pescados**: los dueños de Carrefive también recopilaron qué pescados se vieron involucrados en cada transacción de este producto, tanto de obtención como de uso.
6. **Compra de puntos**: Especifica el cliente que los compró, la fecha de la compra y la cantidad de puntos adquiridos.

### **EJERCICIOS**

**0.** Vamos a ver, por grupos, la estructura de los csv's en los que se encuentran los datos, y después, pensad cómo haríais un mapa relación entre las tablas, qué claves situarías como primarias y cuáles foráneas.

**0.1.** Vamos a crear las tablas y la base de datos en SQL, con la estructura que habíamos pensado.

**1.** Selecciona los clientes de la India y Brasil

**2.** Selecciona los clientes italianos que no se encuentren en las categorías 4 y 5

**3.** Selecciona los clientes europeos que hablen castellano

**4.** Averigua la nacionalidad que más clientes tenga a los que se les puede enviar e-mails

**5.** Numero total de transacciones que consiguen puntos

**6.** Número total de puntos conseguidos 

**7.** Número total de clientes distintos que han utilizado puntos

**8.** Número total de puntos utilizados

**9.** Proporción de transacciones en las que se obtienen puntos al comprar pescado con respecto al total de transacciones en las que se obtienen puntos

**10.** Proporción de transacciones en las que se obtienen puntos al comprar pescado con respecto al total de transacciones

**11** Clientes que han pagado puntos en los últimos 2 meses

**12.** Clientes que han comprado puntos antes pero no en los últimos dos meses

**13.** Clientes del sistema que nunca han comprado puntos

In [2]:
import pandas as pd
import numpy as np
import sqlite3
import glob
import os
import csv

In [3]:
carrefive_cvs=glob.glob('./*.csv')
print(carrefive_cvs)

['.\\Cliente.csv', '.\\Compra_puntos.csv', '.\\Marketing.csv', '.\\Obtencion_puntos.csv', '.\\Pescados.csv', '.\\Uso_de_puntos.csv']


In [4]:
type(carrefive_cvs)

list

In [5]:
conn = sqlite3.connect('carrefive.db')
cursor = conn.cursor()

In [6]:
df_marketing=pd.read_csv('./Marketing.csv')
df_obtencion_pts= pd.read_csv('./Obtencion_puntos.csv')
df_pescado=pd.read_csv('./Pescados.csv')
df_uso_pts=pd.read_csv('./Uso_de_puntos.csv')
df_compra_pts= pd.read_csv('./Compra_puntos.csv')
df_clientes= pd.read_csv('./Cliente.csv')


In [7]:
tables = pd.read_sql(""" select *
                        from sqlite_master 
                        where type='table'
                        order by name;
                        """, conn)
tables

,type,name,tbl_name,rootpage,sql
0,table,CLIENTE,CLIENTE,2,CREATE TABLE CLIENTE (\n /* NULL es un ...
1,table,COMPRA_PUNTOS,COMPRA_PUNTOS,6,CREATE TABLE COMPRA_PUNTOS (\n /* NULL ...
2,table,MARKETING,MARKETING,8,CREATE TABLE MARKETING (\n /* NULL es u...
3,table,OBTENCION_PUNTOS,OBTENCION_PUNTOS,7,CREATE TABLE OBTENCION_PUNTOS (\n /* NU...
4,table,PESCADO,PESCADO,5,CREATE TABLE PESCADO (\n /* NULL es un ...
5,table,USO_PUNTOS,USO_PUNTOS,4,CREATE TABLE USO_PUNTOS (\n /* NULL es ...


In [8]:
cursor.execute(
    """
    /*
    Creamos la tabla en la base de datos
    */
    CREATE TABLE IF NOT EXISTS CLIENTE (
        /* NULL es un valor nulo por defecto
            NOT NULL obliga a que el valor venga informado
        */
        customer_id INT PRIMARY KEY
        ,birthday DATETIME
        ,country VARCHAR(50)
        ,spanish_speaker BOOL
        ,join_date DATETIME
        ,tier VARCHAR(10)
    );
    """
)
# Guardamos los cambios con el comando commmit
conn.commit()

In [9]:
# df_clientes.drop("Unnamed: 0",axis=1,inplace=True)
# cursor.executemany("INSERT INTO CLIENTE VALUES (?,?,?,?,?,?)", df_clientes.values.tolist())
conn.commit()

In [10]:
cursor.execute(
    """
    /*
    Creamos la tabla en la base de datos
    */
    CREATE TABLE IF NOT EXISTS USO_PUNTOS (
        /* NULL es un valor nulo por defecto
            NOT NULL obliga a que el valor venga informado
        */
        transaccion_id INT,
        customer_id INT,
        tipo_transaccion VARCHAR(50),
        fecha_trans DATETIME,
        puntos_utili INT,
        Uso_pts VARCHAR(10),
        FOREIGN KEY (customer_id) REFERENCES CLIENTE(customer_id)
    );
    """
)
# Guardamos los cambios con el comando commit
conn.commit()


In [11]:
cursor.executemany("INSERT INTO USO_PUNTOS VALUES (?,?,?,?,?,?)", df_uso_pts.values.tolist())
conn.commit()

In [12]:
cursor.execute(
    """
    /*
    Creamos la tabla en la base de datos
    */
    CREATE TABLE IF NOT EXISTS PESCADO (
        /* NULL es un valor nulo por defecto
            NOT NULL obliga a que el valor venga informado
        */
        pescado_id INT
        ,transaccion_id INT
        ,tipo_pescado VARCHAR(10)
    );
    """
)
# Guardamos los cambios con el comando commmit
conn.commit()

In [13]:
# cursor.executemany("INSERT INTO PESCADO VALUES (?,?,?)", df_pescado.values.tolist())
conn.commit()

In [14]:
cursor.execute(
    """
    /*
    Creamos la tabla en la base de datos
    */
    CREATE TABLE IF NOT EXISTS COMPRA_PUNTOS (
        /* NULL es un valor nulo por defecto
            NOT NULL obliga a que el valor venga informado
        */
        compra_id INT
        ,customer_id INT 
        ,fecha_compra DATETIME
        ,puntos_comprados INT
        ,FOREIGN KEY (customer_id) REFERENCES CLIENTE(customer_id)
    );
    """
)
# Guardamos los cambios con el comando commmit
conn.commit()

In [15]:
# cursor.executemany("INSERT INTO COMPRA_PUNTOS VALUES (?,?,?,?)", df_compra_pts.values.tolist())
conn.commit()

In [16]:
cursor.execute(
    """
    /*
    Creamos la tabla en la base de datos
    */
    CREATE TABLE IF NOT EXISTS OBTENCION_PUNTOS (
        /* NULL es un valor nulo por defecto
            NOT NULL obliga a que el valor venga informado
        */
        transaccion_id INT
        ,customer_id INT 
        ,tipo_transaccion VARCHAR(50)
        ,fecha_trans DATETIME
        ,puntos_conseguidos INT
        ,obtencion_puntos INT
        ,FOREIGN KEY (customer_id) REFERENCES CLIENTE(customer_id)
    );
    """
)
# Guardamos los cambios con el comando commmit
conn.commit()

In [17]:
# cursor.executemany("INSERT INTO OBTENCION_PUNTOS VALUES (?,?,?,?,?,?)", df_obtencion_pts.values.tolist())
conn.commit()

In [18]:
cursor.execute(
    """
    /*
    Creamos la tabla en la base de datos
    */
    CREATE TABLE IF NOT EXISTS MARKETING (
        /* NULL es un valor nulo por defecto
            NOT NULL obliga a que el valor venga informado
        */
        customer_id INT  
        ,permiso_mail VARCHAR(50)
        ,FOREIGN KEY (customer_id) REFERENCES CLIENTE(customer_id)
    );
    """
)
# Guardamos los cambios con el comando commmit
conn.commit()

In [19]:
# df_marketing.drop("Unnamed: 0",axis=1,inplace=True)
# cursor.executemany("INSERT INTO MARKETING VALUES (?,?)", df_marketing.values.tolist())
conn.commit()


In [20]:
tables = pd.read_sql(""" select *
                        from sqlite_master 
                        where type='table'
                        order by name;
                        """, conn)
tables


,type,name,tbl_name,rootpage,sql
0,table,CLIENTE,CLIENTE,2,CREATE TABLE CLIENTE (\n /* NULL es un ...
1,table,COMPRA_PUNTOS,COMPRA_PUNTOS,6,CREATE TABLE COMPRA_PUNTOS (\n /* NULL ...
2,table,MARKETING,MARKETING,8,CREATE TABLE MARKETING (\n /* NULL es u...
3,table,OBTENCION_PUNTOS,OBTENCION_PUNTOS,7,CREATE TABLE OBTENCION_PUNTOS (\n /* NU...
4,table,PESCADO,PESCADO,5,CREATE TABLE PESCADO (\n /* NULL es un ...
5,table,USO_PUNTOS,USO_PUNTOS,4,CREATE TABLE USO_PUNTOS (\n /* NULL es ...


In [21]:
q = """
    SELECT *
    FROM cliente
    """
df = pd.read_sql(q,conn)
df

,customer_id,birthday,country,spanish_speaker,join_date,tier
0,4,09/25/1989,IN,0,08/18/2020,1. STANDARD
1,33,02/12/1943,AU,1,09/18/2020,5. EXCELSIOR
2,34,01/14/1949,IN,1,01/10/2022,3. GOLD
3,82,05/14/1999,BR,0,06/10/2020,2. BUSINESS
4,96,12/04/1938,IN,0,05/23/2023,3. GOLD
5,98,10/21/1997,AU,1,04/14/2023,3. GOLD
6,104,02/26/1967,IN,1,10/05/2022,4. PREMIUM
7,109,04/16/1937,IT,0,09/10/2022,5. EXCELSIOR
8,117,01/22/1935,FR,1,02/25/2023,1. STANDARD
9,120,01/01/1940,US,0,04/14/2022,3. GOLD


**1.** Selecciona los clientes de la India y Brasil

In [22]:
q= """
    SELECT *
    FROM cliente
    WHERE country = 'BR'
    OR country = 'IN'

"""

df = pd.read_sql(q,conn)
df

,customer_id,birthday,country,spanish_speaker,join_date,tier
0,4,09/25/1989,IN,0,08/18/2020,1. STANDARD
1,34,01/14/1949,IN,1,01/10/2022,3. GOLD
2,82,05/14/1999,BR,0,06/10/2020,2. BUSINESS
3,96,12/04/1938,IN,0,05/23/2023,3. GOLD
4,104,02/26/1967,IN,1,10/05/2022,4. PREMIUM
5,205,09/12/1950,IN,0,08/23/2022,4. PREMIUM
6,210,10/26/1997,IN,0,05/23/2021,2. BUSINESS
7,331,02/02/1964,IN,0,03/18/2022,2. BUSINESS
8,493,09/26/1962,IN,0,05/10/2023,1. STANDARD
9,505,09/27/1963,BR,1,10/09/2020,2. BUSINESS


**2.** Selecciona los clientes italianos que no se encuentren en las categorías 4 y 5

** Categorías ** 1. Standar; 4. PREMIUN; 5. EXCELSIUR

In [23]:
q= """
    SELECT tier
    FROM cliente
    WHERE country = 'IT'
    
   
"""

df = pd.read_sql(q,conn)
df

,tier
0,5. EXCELSIOR
1,4. PREMIUM
2,4. PREMIUM
3,5. EXCELSIOR
4,1. STANDARD
5,5. EXCELSIOR
6,1. STANDARD


In [24]:
q= """
    SELECT *
    FROM cliente
    WHERE country = 'IT'
    AND tier != '5. EXCELSIOR'
    AND tier != '4. PREMIUM'

"""

df = pd.read_sql(q,conn)
df

,customer_id,birthday,country,spanish_speaker,join_date,tier
0,412,10/17/1973,IT,1,04/18/2022,1. STANDARD
1,600,09/03/1948,IT,0,05/17/2023,1. STANDARD


**3.** Selecciona los clientes europeos que hablen castellano

In [25]:
q= """
    SELECT *
    FROM cliente
    WHERE spanish_speaker = '1'
    AND( country = 'IT'
    OR country = 'FR'
    OR country = 'SP'
    OR country = 'DE'
    )
"""
df = pd.read_sql(q,conn)
df

,customer_id,birthday,country,spanish_speaker,join_date,tier
0,117,01/22/1935,FR,1,02/25/2023,1. STANDARD
1,129,12/11/1948,SP,1,06/24/2020,2. BUSINESS
2,177,12/28/1966,SP,1,06/06/2021,3. GOLD
3,187,03/24/1937,IT,1,11/15/2021,4. PREMIUM
4,188,01/04/1987,DE,1,03/12/2023,1. STANDARD
5,233,04/02/1948,FR,1,04/10/2022,5. EXCELSIOR
6,253,04/06/1985,IT,1,05/03/2022,5. EXCELSIOR
7,290,01/17/1952,FR,1,04/10/2022,2. BUSINESS
8,412,10/17/1973,IT,1,04/18/2022,1. STANDARD
9,545,04/09/1977,SP,1,09/04/2021,4. PREMIUM


**4.** Averigua la nacionalidad que más clientes tenga a los que se les puede enviar e-mails

In [26]:
q = """
    SELECT *
    FROM marketing
    WHERE permiso_mail = 'Y'
    
    """
df = pd.read_sql(q,conn)
df

,customer_id,permiso_mail
0,33,Y
1,109,Y
2,120,Y
3,188,Y
4,205,Y
5,210,Y
6,219,Y
7,233,Y
8,249,Y
9,253,Y


In [27]:
q = """
    SELECT COUNT (country),
    country
    FROM cliente
    INNER JOIN marketing ON cliente.customer_id = marketing.customer_id
    WHERE permiso_mail = 'Y'
    GROUP BY country
    """
df = pd.read_sql(q,conn)
df

,COUNT (country),country
0,4,AU
1,2,BR
2,1,DE
3,3,FR
4,4,IN
5,3,IT
6,1,JP
7,1,SP
8,2,UK
9,2,US


**5.** Numero total de transacciones que consiguen puntos

In [28]:
q = """
    SELECT COUNT (obtencion_puntos)
    FROM obtencion_puntos
    
    
    """
df = pd.read_sql(q,conn)
df

,COUNT (obtencion_puntos)
0,50


**6.** Número total de puntos conseguidos 

In [29]:
q = """
    SELECT SUM(puntos_conseguidos), puntos_conseguidos
    FROM obtencion_puntos
    
    
    """
df = pd.read_sql(q,conn)
df

,SUM(puntos_conseguidos),puntos_conseguidos
0,500,2


**7.** Número total de clientes distintos que han utilizado puntos


In [39]:
q = """
    SELECT COUNT (DISTINCT customer_id)
    FROM uso_puntos
    
    
    """
df = pd.read_sql(q,conn)
df

,COUNT (DISTINCT customer_id)
0,32


**8.** Número total de puntos utilizados



In [48]:
q = """
    SELECT SUM(puntos_utili)
    FROM uso_puntos
    
    
    """
df = pd.read_sql(q,conn)
df

,SUM(puntos_utili)
0,1278


**9.** Proporción de transacciones en las que se obtienen puntos al comprar pescado con respecto al total de transacciones en las que se obtienen puntos

In [64]:
q = """
SELECT COUNT(*) AS total_transaccion,
       COUNT(*) * 100.0 / (SELECT COUNT(*) FROM obtencion_puntos) AS proporcion
FROM obtencion_puntos
WHERE obtencion_puntos = 'Pescado';

    """
df = pd.read_sql(q,conn)
df

,total_transaccion,proporcion
0,10,20.0


**10.** Proporción de transacciones en las que se obtienen puntos al comprar pescado con respecto al total de transacciones

In [65]:
q = """
SELECT COUNT(*) AS total_transaccion,
       COUNT(*) * 100.0 / (SELECT COUNT(*) FROM obtencion_puntos) AS proporcion
FROM obtencion_puntos
WHERE obtencion_puntos = 'Pescado';

    """
df = pd.read_sql(q,conn)
df

,total_transaccion,proporcion
0,10,20.0


**11** Clientes que han pagado puntos en los últimos 2 meses

In [72]:
q = """
SELECT *
FROM compra_puntos
LIMIT 1

    """
df = pd.read_sql(q,conn)
df

,compra_id,customer_id,fecha_compra,puntos_comprados
0,100896,4,05/15/2023,50


In [73]:
# Cambiamos la fecha utilizando los substrato - leer bien su uso y aplicación / con el DATE_FORMAT no salio bien y tampoco renombrando la columna 

q= '''
SELECT *
FROM Compra_puntos
WHERE strftime('%Y-%m-%d', substr(fecha_compra, 7, 4) || '-' || substr(fecha_compra, 1, 2) || '-' || substr(fecha_compra, 4, 2))
> strftime('%Y-%m-%d', 'now', '-2 months')
LIMIT 10
'''
df = pd.read_sql(q,conn)
df



,compra_id,customer_id,fecha_compra,puntos_comprados
0,100896,4,05/15/2023,50
1,100983,82,04/11/2023,20
2,100537,96,05/01/2023,15
3,100262,120,04/07/2023,40
4,100741,177,04/13/2023,20
5,100658,187,04/15/2023,30
6,100563,188,05/04/2023,50
7,100236,210,05/24/2023,10
8,100201,247,04/25/2023,20
9,100426,253,04/17/2023,5


**12.** Clientes que han comprado puntos antes pero no en los últimos dos meses

In [75]:
# Limpiamos los clientes que se repiten basados en el customer_id y analizamos la última fecha de compra de puntos y si es menor a dos meses. Volvemos a usar el 'substr' para modificar el formato fecha.

q='''

SELECT customer_id, fecha_compra, puntos_comprados
FROM Compra_puntos
WHERE customer_id IN (
    SELECT DISTINCT customer_id
    FROM Compra_puntos
    WHERE strftime('%Y-%m-%d', substr(fecha_compra, 7, 4) || '-' || substr(fecha_compra, 4, 2) || '-' || substr(fecha_compra, 1, 2))
    < strftime('%Y-%m-%d', 'now', '-2 months')
) AND customer_id NOT IN (
    SELECT DISTINCT customer_id
    FROM Compra_puntos
    WHERE strftime('%Y-%m-%d', substr(fecha_compra, 7, 4) || '-' || substr(fecha_compra, 4, 2) || '-' || substr(fecha_compra, 1, 2))
    >= strftime('%Y-%m-%d', 'now', '-2 months')
)
'''
df = pd.read_sql(q,conn)
df


,customer_id,fecha_compra,puntos_comprados
0,96,05/01/2023,15
1,219,02/02/2023,30
2,401,04/02/2023,30
3,538,01/03/2023,30
4,597,04/03/2023,15
